In [ ]:
import numpy as np
import pandas as pd
import itertools
import h5py
from astropy.stats import sigma_clip
import matplotlib.pyplot as plt
from scipy import stats
from scipy import special
from scipy import integrate
from scipy import interpolate
from scipy import linalg
from scipy import signal
from scipy.optimize import curve_fit
import time
from pathlib import Path
import os
import random
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel
from typing import Tuple

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 120
#plt.rcParams['text.usetex'] = True

import sys
import mpld3

sys.path.insert(0, '../../')

from modules import prh_mc_utils as pmu

In [ ]:
file_path = Path('../../data/cnn_base_data/original_data/HE0435_Bonvin2016.rdb_.txt')

qso_id = file_path.name.split('_')[0]
qso_data = pd.read_table(file_path)

t = qso_data['mhjd'].to_numpy(dtype=np.float64)
A = qso_data['mag_A'].to_numpy(dtype=np.float64)
B = qso_data['mag_B'].to_numpy(dtype=np.float64)
C = qso_data['mag_C'].to_numpy(dtype=np.float64)
D = qso_data['mag_D'].to_numpy(dtype=np.float64)
errA = qso_data['magerr_A'].to_numpy(dtype=np.float64)
errB = qso_data['magerr_B'].to_numpy(dtype=np.float64)
errC = qso_data['magerr_C'].to_numpy(dtype=np.float64)
errD = qso_data['magerr_D'].to_numpy(dtype=np.float64)

In [ ]:
plt.scatter(t, A, label=f'{qso_id} A')
plt.scatter(t, B, label=f'{qso_id} B')
plt.scatter(t, C, label=f'{qso_id} C')
plt.scatter(t, D, label=f'{qso_id} D')
plt.legend()

In [ ]:
dts = np.diff(qso_data['mhjd'])

new_dts = sigma_clip(dts, masked=False, cenfunc='mean')

N_sampl_mean = (t[-1] - t[0])/new_dts.mean()

print(N_sampl_mean)

In [ ]:
image1 = 'A'
image2 = 'B'

mag1 = qso_data[f'mag_{image1}'].values
mag2 = qso_data[f'mag_{image2}'].values

magerr1 = qso_data[f'magerr_{image1}'].values
magerr2 = qso_data[f'magerr_{image2}'].values

y_input, err_input = pmu.mags_to_fluxsum_mag(mag1, mag2, magerr1, magerr2)

In [ ]:
kernel = ConstantKernel(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)

gp = GaussianProcessRegressor(kernel=kernel, alpha=err_input**2, n_restarts_optimizer=10, 
                              optimizer='fmin_l_bfgs_b', normalize_y=True)

gp.fit(np.expand_dims(t,1), y_input)

N = 2000
support = np.linspace(t[0], t[-1], N)

y, sigma = gp.predict(np.expand_dims(support, 1), return_std=True)

In [ ]:
plt.scatter(t, y_input, color='b')
plt.plot(support, y, color='b')
plt.fill_between(support, y - 1.96*sigma, y + 1.96*sigma, fc='b', alpha=0.5)

In [ ]:
tau, v = pmu.estimate_structure_func_from_data(support, y, sigma, n_bins=50)

In [ ]:
mask = v>=0
tau = tau[mask]
v = v[mask]

max_lag = 0.40*tau[-1]
mask = tau <= max_lag

v = v[mask]
tau = tau[mask]

In [ ]:
pars = stats.linregress(np.log10(tau), np.log10(v))
slope = pars.slope
intercept = pars.intercept

plt.scatter(tau, v, color='blue')
plt.plot(tau, 10**intercept * tau**slope, color='red')
plt.xscale('log')
plt.yscale('log')
plt.grid()

In [ ]:
delay = 200
t_doubled = np.concatenate([support, support - delay])
lags = pmu.compute_lags_matrix(t_doubled)

C = ((y-sigma)**2).mean() - pmu.power_law_sf(lags, slope, intercept)

L = np.linalg.cholesky(C)

y_out = L @ np.random.normal(0, 1, 2*N)

plt.plot(y_out[:N])
plt.plot(y_out[N:])